# Examples for Behavior Cloning Dataset Generation and Usage in Training

This notebook contains convenient examples demonstrating use of dataset generation and training pipeline functions.

1. Start off by running the first cell below.
2. Then, proceed in order. Some cells may be able to be skipped depending on their content.

In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys
from pathlib import Path
sys.path.append(str(Path(os.getcwd()).parent.absolute()))

### Generating a Bimanual Behavior Cloning Dataset

Here, we generate datasets of (BimanualObs, BimanualAction) samples for the "pickup block" and "pass block" task using custom, privileged-knowledge policies.
This takes a while, so I've implemented convenient mechanisms for resuming data generation between sessions.
The resulting output files can be accessed again with the BimanualDataset class, demonstrated in the following section.

##### *Pickup block* dataset

In [ ]:
from policy.privileged_pickup_block_policy import PrivilegedPickupBlockPolicy
from robot.sim import BimanualSim
from train.dataset import generate_bimanual_dataset
from validate.evaluation import PickupBlockTaskEvaluator

pickup_block_dataset_path = Path('D:/bimaminobolonana/pickup-block-dataset')
generate_bimanual_dataset(
  save_dir=pickup_block_dataset_path,
  create_sim=lambda: BimanualSim(merge_xml_files=['block.xml'], camera_dims=(128, 128)),
  create_privileged_policy=PrivilegedPickupBlockPolicy,
  create_task_evaluator=PickupBlockTaskEvaluator,
  total_sample_count=600,
  max_steps_per_rollout=600,
  skip_frames=0,
  camera_dims=(128, 128),
  resume=True
)

##### *Pass block* dataset

In [ ]:
from policy.privileged_pass_block_policy import PrivilegedPassBlockPolicy
from robot.sim import BimanualSim, randomize_block_position
from train.dataset import generate_bimanual_dataset
from validate.evaluation import PassBlockTaskEvaluator

pass_block_dataset_path = Path('D:/bimaminobolonana/pass-block-dataset')
generate_bimanual_dataset(
  save_dir=pass_block_dataset_path,
  create_sim=lambda: BimanualSim(merge_xml_files=['block.xml'], camera_dims=(128, 128), on_mujoco_init=randomize_block_position),
  create_privileged_policy=PrivilegedPassBlockPolicy,
  create_task_evaluator=PassBlockTaskEvaluator,
  total_sample_count=10000,
  max_steps_per_rollout=600,
  skip_frames=2,
  camera_dims=(128, 128),
  resume=True
)

### Viewing a Rollout from the Generated Dataset

In [4]:
from robot.visualize import save_frames_to_video
from train.dataset import BimanualDataset
pickup_block_dataset_path = Path('/mnt/data/simple-pickup-no-randomization')
dataset = BimanualDataset(pickup_block_dataset_path)
rollout_number = 0
rollout_length = dataset.metadata.rollout_lengths[rollout_number]
rollout_start = sum(dataset.metadata.rollout_lengths[:rollout_number])
observations = [dataset[i][0] for i in range(rollout_start, rollout_start + rollout_length)]

os.makedirs('out', exist_ok=True)
left_wrist_video_path = f'out/left_wrist_rollout_{rollout_number}.mp4'
right_wrist_video_path = f'out/right_wrist_rollout_{rollout_number}.mp4'
save_frames_to_video([observation.visual[0, 0].detach().numpy() for observation in observations], left_wrist_video_path)
save_frames_to_video([observation.visual[0, 1].detach().numpy() for observation in observations], right_wrist_video_path)
print(f"Saved left wrist video to {left_wrist_video_path}")

Saved left wrist video to out/left_wrist_rollout_0.mp4


In [5]:
from IPython.display import Video
Video(left_wrist_video_path, width=400, height=400)

In [19]:
from IPython.display import Video
Video(right_wrist_video_path, width=400, height=400)

### Training Pipeline

Here we demonstrate the suite of utilities in this `train` subpackage.

1. `train.train_utils.Logs`
  - Utility class for organizing training output files. We should use this going forward to remain organized and expand its capabilities as needed.
2. `train.dataset.BimanualDataset`
  - Dataset implementation for reading behavior cloning data.
3. `train.trainer.BCTrainer`
  - A class for training an arbitrary model for behavior cloning. We should more-or-less keep this class's core logic as-is.

Each of these utilities currently have some TODO comments in their code - take a look. In summary, (1. Logs) doesn't currently do much logging - the Jobs class in the same file needs to have log piping implemented. (2. Dataset) currently just uses robot.sim numpy-based dataclasses for observations and actions, it should start using the torch-based versions defined in dataset.py and moving the .npy files to GPU. (3. BCTrainer) isn't very configurable right now - the optimizer and other details should be constructor args that we can dynamically configure with hydra or something.

In [ ]:
import os
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from robot.sim import JOINT_OBSERVATION_SIZE
from train.dataset import BimanualDataset, TensorBimanualAction, TensorBimanualObs
from train.train_utils import Logs
from train.trainer import BCTrainer, BimanualActor

# example bimanual actor class
class ExampleModel(BimanualActor):
  def __init__(self, observation_size: int, action_size: int, hidden_size: int = 256):
    super().__init__()
    self.layers = nn.Sequential(
      nn.Linear(observation_size, hidden_size),
      nn.ReLU(),
      nn.Linear(hidden_size, hidden_size),
      nn.ReLU(),
      nn.Linear(hidden_size, action_size)
    )

  def forward(self, obs: TensorBimanualObs) -> TensorBimanualAction:
    x = torch.cat((obs.visual.reshape(obs.visual.shape[0], -1), obs.qpos.array), dim=-1)
    return TensorBimanualAction(self.layers(x))

# load dataset and set up trainer
BATCH_SIZE = 256
EPOCHS = 10
CHECKPOINT_FREQUENCY = 1
dataset = BimanualDataset(pickup_block_dataset_path)
dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, collate_fn=BimanualDataset.collate_fn)
os.makedirs('out/training-output', exist_ok=True)
logs = Logs('out/training-output')
new_job = logs.create_new_job(tag='example')

# instantiate model
input_size = dataset.metadata.observation_size - JOINT_OBSERVATION_SIZE  # exclude qvel observation
output_size = dataset.metadata.action_size
model = ExampleModel(input_size, output_size)
if torch.cuda.is_available():
  print('Using CUDA.')
  model = model.cuda()
else:
  print('Using CPU.')

# train with behavior cloning objective
trainer = BCTrainer(dataloader, checkpoint_frequency=CHECKPOINT_FREQUENCY, job=new_job)
trainer.train(model, EPOCHS)

Using CUDA.
Training model for 10 epochs.


Epoch 0: 100%|██████████| 40/40 [02:07<00:00,  3.18s/it]


 - Epoch 0 loss: 504.3760


Epoch 1: 100%|██████████| 40/40 [00:01<00:00, 31.21it/s]


 - Epoch 1 loss: 15.9850


...

 - Epoch 7 loss: 9.4972


Epoch 8: 100%|██████████| 40/40 [00:01<00:00, 35.10it/s]


 - Epoch 8 loss: 8.9548


Epoch 9: 100%|██████████| 40/40 [00:01<00:00, 34.40it/s]


 - Epoch 9 loss: 9.2741
